<a href="https://colab.research.google.com/github/yuxiao-ashes339/My-Wedding-Optimizer/blob/main/WeddingOptimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 51.2 MB/s eta 0:00:00


In [116]:
from pulp import LpMaximize, LpProblem, LpVariable
import numpy as np

In [117]:
# Wedding Cost Optimizer
# Initial data estimated from https://ininkweddings.com/average-cost-of-a-wedding-in-austin/
item_dict = {
    'Planner':[8000,1], #avg price and score
    'Event Coordinator':[2500,1],
    'Venue':[7000,1],
    'Catering':[50,1],# per person
    'Alcohol':[30,1],# per person
    'Photographer':[7000,1],
    'Videographer': [4000,1],
    'DJ':[2500,1],
    'Band':[8000,1],
    'Rentel':[4000,1],
    'Beauty':[800,1],
    'Cake':[6,1],# per person
    'Day-of Paper/Signage':[3000,1],
    'Invitation':[5,1],# per person
    'Officiant':[500,1],
    'Florist':[7000,1],
    'Dress':[5000,1],
    'Attire':[2000,1]
}
initial_constraints_MULT = 1.1 # can be +/- 10% wgt from initial

In [118]:
def BudgetOptimizer(customize_importance_rank, budget, known_spend, guest_num):
  for key in ['Catering','Alcohol','Cake','Invitation']:
    item_dict[key][0] = item_dict[key][0]*guest_num
  remaining_budget = budget - sum(known_spend.values())
  remaing_item = list(set(customize_importance_rank) - set(known_spend.keys()))
  filtered_item_dict = {key: item_dict[key] for key in remaing_item if key in item_dict}
  # normalize and adjust score based on ranking
  wgts = [v[0] for v in filtered_item_dict.values()]
  filtered_item_dict = {
      key: [v[0]/sum(wgts)] + [(len(customize_importance_rank)-customize_importance_rank.index(key))]
      for key, v in filtered_item_dict.items()
  }

  adjusted_constraints_MULT = initial_constraints_MULT*min(len(filtered_item_dict)/len(item_dict),1.5)# adjust constraint based on number of items

  model = LpProblem(name='dynamic_lp',sense=LpMaximize)
  variables = {name: LpVariable(name, lowBound=0) for name in filtered_item_dict.keys()}
  model += np.dot(list(variables.values()), [v[1] for v in filtered_item_dict.values()]), 'Objective'
  model += (sum(list(variables.values()))==1, 'Constraint 1')
  constraint_num = 2
  for name in variables.keys():
    model += (variables[name]<=min(filtered_item_dict[name][0]*adjusted_constraints_MULT,filtered_item_dict[name][0]+0.05), f'Constraint {constraint_num}')
    constraint_num += 1
    model += (variables[name]>=max(filtered_item_dict[name][0]*adjusted_constraints_MULT,filtered_item_dict[name][0]-0.05), f'Constraint {constraint_num}')
    constraint_num += 1
    model += (variables[name]*remaining_budget<=item_dict[name][0]*1.2, f'Constraint {constraint_num}')
    constraint_num += 1
    model += (variables[name]*remaining_budget>=item_dict[name][0]*0.8, f'Constraint {constraint_num}')
    constraint_num += 1

  model.solve()

  print('Optimized spending: ')
  for name, wgt in variables.items():
    print(f'{name}: ${wgt.varValue*remaining_budget:.0f}')

In [119]:
# Add your customize items rank from high to low, don't list if not interested
customize_importance_rank = ['Venue','DJ','Dress','Photographer','Videographer','Catering','Beauty','Alcohol','Cake','Florist','Attire','Invitation','Event Coordinator']
# Add your budget
budget = 40000
guest_num = 100
known_spend = {
    'Venue': 4000,
    'DJ': 1500,
    'Photographer': 3000,
    'Videographer':2000,
    'Florist':4250,
    'Dress': 4500
}
BudgetOptimizer(customize_importance_rank, budget,known_spend, guest_num)

Optimized spending: 
Cake: $720
Invitation: $600
Alcohol: $3600
Catering: $6167
Event Coordinator: $3000
Beauty: $960
Attire: $2400
